# Training Notebook for System of ODEs with not constant force function (SOncFF)

In [ ]:
%load_ext autoreload
%autoreload 2 

In [ ]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
import json
import os
from scipy.integrate import solve_ivp
from tqdm.auto import trange

# Add parent directory to sys.path
from pathlib import Path
current_path = Path.cwd()
parent_dir = current_path.parent.parent
sys.path.append(str(parent_dir))

# Import necessary modules
from src.train import run_model
from src.utils_plot import plot_loss_and_all_solution, plot_head_loss, plot_loss_and_single_solution
from src.load_save import save_model

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
def check_versions_and_device():
  # set the device to the GPU if it is available, otherwise use the CPU
  current_dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  !nvidia-smi
  return current_dev

# set a global device variable to use in code
dev = check_versions_and_device()
print(dev)

#  1) Multi Head Training
### Select equation parameter

-   number of head
-   stiffness parameter $\alpha$
-   force function $f$
-   initiales condition $IC$

In [ ]:
equation_name = "SOncFF"
num_heads = 5

alpha_bounds = (0, 10)
def generate_alpha(k, alpha_bounds = alpha_bounds):
  alpha_list = np.zeros(k)
  for i in range(k):
    alpha_list[i] = np.random.uniform(alpha_bounds[0], alpha_bounds[1])
  return alpha_list
alpha_list = generate_alpha(num_heads)
def get_A(alpha):
    return torch.tensor([[2., -1.], [1-alpha, alpha]], device=dev).double() 
A_list = [get_A(i)for i in alpha_list]


IC_list = [torch.tensor([[2.], [4.]], device=dev).double() for _ in range(num_heads)]
# uncomment if you want random IC
def random_IC(x_bound=[0, 2], y_bound=[8, 12]):
    ICx = np.random.uniform(x_bound[0], x_bound[1], 1)
    ICy = np.random.uniform(y_bound[0], y_bound[1], 1)
    return torch.tensor([ICx, ICy], device=dev).double()
#IC_list = [random_IC() for _ in range(num_heads)]


omega_list = [1.0 for i in range(num_heads)]
# uncomment if you want random omega value in the force function 
omega_bounds = (0, np.pi)
def generate_omega(k, omega_bounds = omega_bounds):
  omega_list = np.zeros(k)
  for i in range(k):
    omega_list[i] = np.random.uniform(omega_bounds[0], omega_bounds[1])
  return omega_list
#omega_list = generate_omega(num_heads)
force_list = [
    lambda t, alpha=a, omega=w: torch.vstack([2*torch.sin(omega*t), alpha*(torch.cos(omega*t)-torch.sin(omega*t))]).T.double()
    if not isinstance(t, (float, int))
    else np.array([2*np.sin(omega*t), alpha*(np.cos(omega*t)-np.sin(omega*t))]).T
    for a, w in zip(alpha_list, omega_list)
]

### Select training parameter

-   range of training $x_{range}$
-   activation function
-   number of hidden layer
-   number of equation
-   number of iterations
-   learning rate $lr$
-   sample size during epoch
-   gradient decay

In [ ]:
x_range = [0, 10]
activation = "silu"
hid_lay = list(np.array([124, 124, 132]))
num_equations = 2
iterations = 10
lr = 0.0001
sample_size = 200
decay=True

### Train the multi head model

In [ ]:
verbose = True
loss_hist, trained_model, model_time = run_model(iterations=iterations, x_range=x_range, lr=lr, A_list=A_list, 
                                                     IC_list=IC_list, force=force_list, hid_lay=hid_lay, activation=activation,
                                                     num_equations=num_equations, num_heads=num_heads, sample_size=sample_size,
                                                     decay=decay, dev=dev, verbose=verbose)
# date tag to save
from datetime import datetime
now = datetime.now()
# Format the date and time as a string in the format 'mmddhhmm'
formatted_datetime = now.strftime('%m%d%H%M')
# Convert the formatted string to an integer
formatted_datetime_int = int(formatted_datetime)

### Plot training outcome

In [ ]:
# function to numerically compute the solution to any set of two coupled, linear first-order ODES
def double_coupled_equation(t, y, A, force):
    return np.array([force(t)[..., 0].item()- A[0][1] * y[1] - A[0][0] * y[0],
                     force(t)[..., 1].item() - A[1][0] * y[0] - A[1][1] * y[1]])

numerical_sol_fct = lambda x, v, A, force: (solve_ivp(double_coupled_equation, [x_range[0], x_range[1]],
                                                    v.squeeze(), args=(A, force), t_eval=x.squeeze(), method="Radau").y)

plot_loss_and_all_solution(x_range=x_range, true_functs=numerical_sol_fct,  
                           trained_model=trained_model, IC_list=IC_list, A_list=A_list,
                           force=force_list, train_losses=loss_hist,  device=dev)
plot_head_loss(loss_hist["head"], alpha_list)

### Save model and training history

In [ ]:
model_name = "big_test"

save_model(trained_model, formatted_datetime_int, equation_name, model_name,
           x_range, iterations, hid_lay, num_equations, num_heads, A_list,
           IC_list, force_list, alpha_list, loss_hist)

# 2) Single Head Training

### Select equation parameter

-   stiffness parameter $\alpha$
-   force function $f$
-   initiales condition $IC$

In [ ]:
num_heads = 1
alpha_list = [10, 20, 30, 40]
A_list = [torch.tensor([[2., -1.], [1-i, i]], device=dev).double()  for i in alpha_list]
IC_list = [torch.tensor([[2.], [4.]], device=dev).double()  for _ in alpha_list]
force_list = [
    lambda t, alpha=a, omega=1: torch.vstack([2*torch.sin(omega*t), alpha*(torch.cos(omega*t)-torch.sin(omega*t))]).T.double()
    if not isinstance(t, (float, int))
    else np.array([2*np.sin(omega*t), alpha*(np.cos(omega*t)-np.sin(omega*t))]).T
    for a in alpha_list
]

### Select training parameter

-   number of iterations
-   learning rate $lr$

In [ ]:
lr_list = [0.0001, 0.0001, 0.00001, 0.00001]
iterations_list = [20000, 30000, 60000, 80000]
iterations_list = [100, 100, 100, 100]

### Train single head model

In [ ]:
solution_PINNS = []
rng = np.random.default_rng()
t_eval = torch.arange(x_range[0], x_range[1], 0.001, requires_grad=True, device=dev).double()
t_eval = t_eval[np.concatenate(([0], rng.choice(range(1, len(t_eval)), size=512 - 1, replace=False)))]
t_eval = t_eval.reshape(-1, 1)
t_eval, _ = t_eval.sort(dim=0)

for i in trange(len(alpha_list)):
    if i==0:
        loss_history, trained_model, _ = run_model(iterations=iterations_list[i], x_range=x_range, lr=lr_list[i], A_list=[A_list[i]], 
                                            IC_list=[IC_list[i]], force=[force_list[i]], hid_lay=hid_lay, activation=activation,
                                            num_equations=num_equations, num_heads=num_heads, sample_size=sample_size,
                                            decay=decay, dev=dev, verbose=False)
    else:
        _, _, _ = run_model(iterations=iterations_list[i], x_range=x_range, lr=lr_list[i], A_list=[A_list[i]], 
                            IC_list=[IC_list[i]], force=[force_list[i]], hid_lay=hid_lay, activation=activation,
                            num_equations=num_equations, num_heads=num_heads, sample_size=sample_size,
                             decay=decay, dev=dev, verbose=False)
    solution_PINNS.append(trained_model(t_eval)[0])

### Plot training of the first model

In [ ]:
plot_loss_and_single_solution(x_range=x_range, true_functs=numerical_sol_fct,
                              trained_model=trained_model, IC_list=IC_list, A_list=A_list,
                              force=force_list, train_losses=loss_history, device=dev)

### Plot MAE and MaxAE results over several alpha value

In [ ]:
mae_y1 = []
mae_y2 = []
maxae_y1 = []
maxae_y2 = []

for i in range(len(alpha_list)):
    pinns = solution_PINNS[i].detach().cpu().numpy()
    numerical = numerical_sol_fct(t_eval.detach().cpu().numpy(),
                                  IC_list[0].detach().cpu().numpy(),
                                  A_list[i].detach().cpu().numpy(),
                                  force_list[i]).T
    absolute_error = np.abs(pinns[:, 0, :] - numerical)
    mae_y1.append(absolute_error.mean(0)[0])
    mae_y2.append(absolute_error.mean(0)[1])
    maxae_y1.append(absolute_error.max(0)[0])
    maxae_y2.append(absolute_error.max(0)[1])

fig, ax = plt.subplots(1, figsize=(13, 4))

ax.plot(alpha_list, mae_y1, "-o", label="$MAE$ ${y_1}$", linewidth=2, markersize=6)
ax.plot(alpha_list, mae_y2,"-o", label="$MAE$ ${y_2}$", linewidth=2, markersize=6)
ax.plot(alpha_list, maxae_y1, "-x", color="#1f77b4", label="$MaxAE$ ${y_1}$", linewidth=2, markersize=8)
ax.plot(alpha_list, maxae_y2, "-x", color="#ff7f0e", label="$MaxAE$ ${y_2}$", linewidth=2, markersize=8)

ax.set_yscale("log")
ax.set_title(r"Mean and Max Absolute Error with increasing Stiffness", fontsize=20)
ax.set_xlabel(r'Stiffness parameter $\alpha$', fontsize=16)
ax.set_ylabel('Absolute Error', fontsize=16)
ax.set_xticks(alpha_list, alpha_list)
ax.set_yticks([0.1, 0.01, 0.001],
              [r"$10^{-1}$", r"$10^{-2}$", r"$10^{-3}$"])
ax.grid()
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=16)
ax.legend(loc='best', fontsize=14)

### Save MAE and MaxAE results over several alpha value

In [ ]:
history = {}
history["alpha_list"] = alpha_list
history["mae_y1"] = mae_y1
history["mae_y2"] = mae_y2
history["maxae_y1"] = maxae_y1
history["maxae_y2"] = maxae_y2

current_path = Path.cwd().parent.parent
path = os.path.join(current_path, "result_history")
with open(os.path.join(path, "SOncFF_Error_Trained.json"),  "w") as fp:
    json.dump(history, fp)